In [2]:
import pandas as pd
import numpy as np

#df = pd.read_csv('~/goinfre/data/train_concat2_90_00001.csv')
#y = pd.read_csv('~/Downloads/y2.csv', sep=';', usecols=[1])
df = pd.read_csv('E:\\Downloads\\test_concat_95_00001.csv')
df.apply(pd.to_numeric,downcast='unsigned')
y = pd.read_csv('E:\\Downloads\\y.csv', sep=';', usecols=[1])
y.apply(pd.to_numeric,downcast='unsigned')

df.shape

(13250, 6279)

In [1]:
from sklearn.svm import SVC
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp

param_range = [0.001,0.01,0.1,1,10,100]
#param_range = [1]

cv = StratifiedKFold(n_splits=3, random_state=42)

for i in param_range:
    
    classifier = SVC(C=i, random_state=42)

    aucs = []
    tprs = []
    mean_fpr = np.linspace(0, 1, 100)

    for train_index, test_index in cv.split(df, y):
        model = classifier.fit(df.iloc[train_index], y.iloc[train_index].values.ravel())
        if hasattr(model, "predict_proba"):
            probas_ = model.predict_proba(df.iloc[test_index])[:, 1]
        else:  # use decision function
            probas_ = model.decision_function(df.iloc[test_index])
        fpr, tpr, thresholds = roc_curve(y.iloc[test_index], probas_)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    print('param=%s Mean ROC (AUC = %0.3f)' % (str(i), mean_auc))
    print(aucs)

NameError: name 'np' is not defined